### Building a RAG Pipeline with LangChain, Gemini, and Pinecone
This notebook walks through the end-to-end development of a Retrieval-Augmented Generation (RAG) system using LangChain and Gemini (Google Generative AI). It demonstrates embedding generation with Google Embeddings, vector storage with Pinecone, similarity search with FAISS/Sklearn, document structuring with LangChain’s Document class, and dynamic prompt chaining using LangChain Runnables. A production-ready retriever and query-answering chain are built with search thresholding and structured prompting.

In [117]:
# 🔧 Environment & Warnings
import os
import warnings
from dotenv import load_dotenv  # For loading environment variables

# 🧠 Embeddings & LLMs
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings  # Google Gemini & embeddings
from langchain_huggingface import HuggingFaceEmbeddings  # HuggingFace embeddings

# 🧾 Prompting & Output Parsers
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub  # LangChain Hub for prebuilt prompts

# 📄 Document & Vector Store
from langchain_core.documents import Document
# from langchain_community.vectorstores import FAISS  # FAISS Vector Store
# from langchain_community.docstore.in_memory import InMemoryDocstore  # In-memory docstore for FAISS
from langchain_pinecone import PineconeVectorStore  # Pinecone integration

# 🌲 Pinecone Configuration
from pinecone import Pinecone, ServerlessSpec  # Pinecone core client & serverless setup

# 📊 Utils
from uuid import uuid4  # Unique ID generator for documents
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity  # For similarity computations
import faiss  # Facebook AI Similarity Search (FAISS)
import pprint


load_dotenv()

warnings.filterwarnings('ignore')

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")
os.environ['LANGSMITH_PROJECT'] = os.getenv("LANGSMITH_PROJECT")
os.environ['LANGSMITH_API_KEY'] = os.getenv("LANGSMITH_API_KEY")
os.environ['LANGSMITH_ENDPOINT'] = os.getenv("LANGSMITH_ENDPOINT")
os.environ['LANGSMITH_TRACING'] = os.getenv("LANGSMITH_TRACING")

In [118]:
embeddings_google = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [119]:
documents = ["India is establish player in world cricket",
            "India own the ODI world cup 1983, 2011",
            "India own the T20 world cup 2007, 2025"]

my_query = "About which country we are talking here?"

document_embedding = embeddings_google.embed_documents(documents)

In [120]:
len(document_embedding[0])

768

In [121]:
my_query_embed = embeddings_google.embed_query(my_query)

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [122]:
cosine_similarity([my_query_embed], document_embedding)

array([[0.57796012, 0.57769502, 0.57979686]])

In [123]:
euclidean_distances([my_query_embed], document_embedding)

array([[0.91873741, 0.9190259 , 0.91673598]])

In [124]:
pinecone_api_key = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [125]:
index_name = 'learn-agenticaiv2'
pc.has_index(index_name)

True

In [126]:
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

In [127]:
pc.has_index(index_name)

True

In [128]:
index_name = pc.Index(index_name)

pinecone_vs = PineconeVectorStore(index=index_name, embedding=embeddings_google)

In [129]:
document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},#additional info
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
documents

[Document(metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
 Document(metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic application

In [130]:
len(documents)

10

In [131]:
for _ in range(len(documents)):
    print(_)
    print(str(uuid4()))

0
5bc86b2e-9e1f-4d58-8fb5-8d76f6af155c
1
e0f66e90-0342-46e5-98e5-6cf6bbe22cbc
2
fd49cc8e-17f8-458f-860f-92efd1bef57e
3
d3bbc134-4ac5-43fe-9dde-dd467a645757
4
a2d7ca04-db3a-44ee-8e27-3345eb961f3c
5
ab68f695-a5f7-44ea-9376-9442fff72a5c
6
2dd90d10-59ff-45b6-9873-a0f8f15c9f17
7
b08add5b-e9c3-4896-88fd-24953b3c1391
8
12ab8500-12fc-4e7e-98ac-5a3d26adf83c
9
f74f99ca-75e5-49da-9f69-79915190db6d


In [132]:
uuids = [str(uuid4()) for _ in range(len(documents))]
uuids

['1d0b565f-d009-4c20-8747-915ec10fa36d',
 '4447d099-628f-4e95-a076-c59ecc0ba6a6',
 'd16daaca-841a-453b-80ce-79a0fef691f7',
 '301a079b-5aff-4567-8150-01ad51dc5108',
 'f71d5a48-a39d-4331-8d6a-07bbeda339ad',
 'ebc3e172-6ac6-42bf-954e-6b241c1fe8c5',
 '48fed8b7-840b-4e45-b0e1-e685742cf4c8',
 '0aa0a99e-4bc0-4353-98a0-c7b318b2cdc4',
 '5acd9be4-cc04-4c35-a2e4-20a393031dcd',
 'de147a83-d20e-48cc-8889-f9d8530d8803']

In [133]:
pinecone_vs.add_documents(documents=documents, id=uuids)

['1f6e061d-725c-4e0d-b73e-703f62ce46f2',
 'f7f19acb-b36f-4d8b-90dd-3e23f4e9eecf',
 '4ab37b63-5aa4-4671-ac20-a1832c63db59',
 '1efba696-5776-411b-a86f-f89fc1d81312',
 'ce41899f-f1c2-4ee1-8136-fca81c97fb61',
 '5ef13e30-f57b-4e6c-b183-edc25bd13947',
 '7a453b6e-ba35-4cf4-991f-627eaaf2d350',
 'c6ee7340-070f-4732-b903-872429f79aa7',
 '348476d2-6c1e-4269-b620-61d32eeead83',
 'cd5e4921-dd92-4313-ad01-12b78ad8a617']

In [134]:
results = pinecone_vs.similarity_search("what langchain provides to us?", k=3)
results

[Document(id='4ab37b63-5aa4-4671-ac20-a1832c63db59', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='687e1025-e918-4c79-a242-be4edf4d6652', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='33e18021-5768-43a7-87be-eebf7d506c07', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!')]

In [135]:
retriever = pinecone_vs.as_retriever(search_type='similarity_score_threshold',
                                     search_kwargs={'score_threshold': 0.7})

In [136]:
retriever.invoke("what is langchain?")

[Document(id='076b8862-6b0b-411a-b373-d2894d905d03', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='33e18021-5768-43a7-87be-eebf7d506c07', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='687e1025-e918-4c79-a242-be4edf4d6652', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='4ab37b63-5aa4-4671-ac20-a1832c63db59', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!')]

In [137]:
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [138]:
prompt = hub.pull('rlm/rag-prompt')

pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [139]:
prompt=PromptTemplate(
    template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""",
    input_variables=['context', 'question']
)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:")

In [140]:
prompt.invoke({'question':'what is langchain', 'context':'langchain is very super framework for LLM.'})

StringPromptValue(text="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: what is langchain \nContext: langchain is very super framework for LLM. \nAnswer:")

In [141]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [142]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [143]:
rag_chain.invoke("what you know?")

"I don't know.  The provided text only expresses a feeling of impending deletion, not any factual knowledge."